<a href="https://colab.research.google.com/github/icaroperetti/artificial-intelligence/blob/main/redes-neurais/rna_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Connect to google drive
from google.colab import drive
drive.mount('/content/drive',force_remount=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import Normalizer
from sklearn import metrics

#Path of the file in my drive
# https://www.kaggle.com/datasets/somesh24/spambase?select=spambase_csv.csv  - E-mail spam

file_path = 'https://raw.githubusercontent.com/icaroperetti/spam-kaggle-csv/main/spambase_csv.csv'


# load the dataset
dataset = loadtxt(file_path,delimiter=",")

X = dataset[:,0:57] # Dados
y = dataset[:,57] # Classe 0 - Não é spam  1 - É spam

print('Before',X)

X = Normalizer().fit(X).transform(X)

print('After',X)

print(X.shape,y.shape)

# print(X,'\n\n',y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=80, shuffle=True)


model = Sequential()
model.add(Dense(100, input_dim=X.shape[1], activation='relu'))
model.add(Dropout(0.2)) 
model.add(Dense(33, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(11, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


# fit the keras model on the dataset
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200, batch_size=100)


loss, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))
print('Loss: %.2f' % (loss*100))
